In [1]:
import panel as pn
import pandas as pd
import sqlite3
from functools import partial
pn.extension()

In [2]:
# product
product_id = pn.widgets.IntInput(name='Product ID')
product_name = pn.widgets.TextInput(name='Product Name')
vendor_cost = pn.widgets.FloatInput(name='Vendor Cost')
customer_cost = pn.widgets.FloatInput(name='Customer Cost')

# customer 
customer_id = pn.widgets.IntInput(name='Customer ID')
first_name = pn.widgets.TextInput(name='First Name')
last_name = pn.widgets.TextInput(name='Last Name')
Phone_C = pn.widgets.TextInput(name='Phone Number')
city = pn.widgets.TextInput(name='City')
zip_code = pn.widgets.IntInput(name='Zip Code')

# employees
employee_id = pn.widgets.IntInput(name='Employee ID')
FirstName = pn.widgets.TextInput(name='First Name')
LastName = pn.widgets.TextInput(name='Last Name')
Phone_E = pn.widgets.TextInput(name='Phone Number')
Job_title = pn.widgets.TextInput(name='Job Title')

# orders
order_id = pn.widgets.IntInput(name='Order ID')
order_date = pn.widgets.DatePicker(name='Order Date')
quantity = pn.widgets.IntInput(name='Quantity')

In [3]:
product = pn.Column(product_id, product_name, vendor_cost, customer_cost)
customer = pn.Column(customer_id, first_name, last_name,Phone_C, city, zip_code)
employees = pn.Column(employee_id, FirstName, LastName,Phone_E, Job_title)
orders = pn.Column(order_id, product_id, customer_id, employee_id, order_date, quantity)

In [4]:
predefined_query = pn.widgets.Button(name=''' SELECT Product_Name from Product JOIN Orders ON Orders.Product_ID=Product.Product_ID ''', button_type='primary')

delete_button = pn.widgets.Button(name='Delete entry', button_type='danger')
#delete_query = pn.widgets.TextInput(value='Ready')

update_button = pn.widgets.Button(name='Update entry', button_type='warning')
#update_query = pn.widgets.TextInput(value='Ready')

insert_button = pn.widgets.Button(name='Insert entry', button_type='success')
#insert_query = pn.widgets.TextInput(value='Ready')
Title_button = pn.widgets.Button(name='Jewelry Business', button_type='primary')
table_options = pn.widgets.RadioBoxGroup(name='RadioBoxGroup', options=['Product', 'Customer','Employees', 'Orders'], inline=False)

In [5]:
con = sqlite3.connect("DBMS.db")

# Load the data into a DataFrame
#df = pd.read_sql_query("SELECT * from Orders", con)
#df

In [6]:
table = pn.widgets.DataFrame(pd.read_sql_query("SELECT * from Product", con), name ='Table')
def table_event(event):
    if table_options.value == 'Product':
        df = pd.read_sql_query("SELECT * from Product", con)
        table.value = df
    elif table_options.value == 'Customer':
        df = pd.read_sql_query("SELECT * from Customer", con)
        table.value = df
    elif table_options.value == 'Employees':
        df = pd.read_sql_query("SELECT * from Employees", con)
        table.value = df
    elif table_options.value == 'Orders':
        df = pd.read_sql_query("SELECT * from Orders", con)
        table.value = df

In [7]:
table_options.param.watch(table_event, 'value')

Watcher(inst=RadioBoxGroup(options=['Product', 'Customer', ...], value='Product'), cls=<class 'panel.widgets.select.RadioBoxGroup'>, fn=<function table_event at 0x7fabb80d9e50>, mode='args', onlychanged=True, parameter_names=('value',), what='value', queued=False, precedence=0)

In [8]:
#product_table = pn.widgets.DataFrame(df, name='Product DataFrame')

In [9]:
#product_table.value

In [10]:
def insert_event(event):
    
    con = sqlite3.connect("DBMS.db")
    
    if table_options.value == 'Product':
        entry = {'Product_ID': product_id.value, 'Product_Name': product_name.value, 'Vendor_Cost': vendor_cost.value, 
                 'Customer_Cost': customer_cost.value}
        #full_table = pd.read_sql_query("SELECT * from product", con)
        #df = pd.DataFrame.from_dict([entry])
    
        #results = pd.concat([full_table, df]).drop_duplicates(subset=['Product_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        cur = con.cursor()
        insert = """INSERT INTO Product
            (Product_ID, Product_Name, Vendor_Cost, Customer_Cost) 
            VALUES (?, ?, ?, ?);"""
        data = (product_id.value, product_name.value, vendor_cost.value, customer_cost.value)
        cur.execute(insert, data)
        con.commit()
        #results.to_sql("Product", con, if_exists="replace", index=False, index_label='Product_ID')
        full_table = pd.read_sql_query("SELECT * from product", con)

        table.value = full_table
        
    elif table_options.value == 'Customer':
        entry = {'Customer_ID': customer_id.value, 'First_Name': first_name.value, 'Last_Name': last_name.value, 'Phone': Phone_C.value,
                'City': city.value, 'Zipcode': zip_code.value}
        full_table = pd.read_sql_query("SELECT * from Customer", con)
    
        df = pd.DataFrame.from_dict([entry])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Customer_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Customer", con, if_exists="replace", index=False, index_label='Customer_ID')
        table.value = results
        full_table = pd.read_sql_query("SELECT * from Customer", con)
        
    elif table_options.value == 'Employees':
        entry = {'Employee_ID': employee_id.value, 'First_Name': FirstName.value, 'Last_Name': LastName.value, 'Phone': Phone_E.value,
                'Job_title': Job_title.value}
        full_table = pd.read_sql_query("SELECT * from Employees", con)
    
        df = pd.DataFrame.from_dict([entry])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Employee_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Employees", con, if_exists="replace", index=False, index_label='Employee_ID')
        table.value = results
        full_table = pd.read_sql_query("SELECT * from Employees", con)
    
    elif table_options.value == 'Orders':
        entry = {'Order_ID': order_id.value, 'Product_ID': product_id.value, 'Customer_ID': customer_id.value, 'Employee_ID': employee_id.value,
                'Order_date': order_date.value, 'Quantity': quantity.value}
        full_table = pd.read_sql_query("SELECT * from Orders", con)
    
        df = pd.DataFrame.from_dict([entry])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Order_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Orders", con, if_exists="replace", index=False, index_label='Order_ID')
        table.value = results
        full_table = pd.read_sql_query("SELECT * from Orders", con)
        
insert_button.on_click(insert_event)


Watcher(inst=Button(button_type='success', name='Insert entry'), cls=<class 'panel.widgets.button.Button'>, fn=<function insert_event at 0x7fabd3db5700>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [11]:
def delete_event(event):
    
    con = sqlite3.connect("DBMS.db")
    if table_options.value == 'Product':
        
        #delete = {'Product_ID': product_id.value, 'Product_Name': product_name.value, 'Vendor_Cost': vendor_cost.value, 'Customer_Cost': (vendor_cost.value + (vendor_cost.value * 5 / 100))}
        delete = {'Product_ID': product_id.value}
        #full_table = pd.read_sql_query("SELECT * from product", con)
        #df = pd.DataFrame.from_dict([delete])
        #results = pd.concat([full_table, df]).drop_duplicates(subset=['Product_ID'], keep=False).reset_index(drop=True)
        cur = con.cursor()
        delete = 'DELETE FROM Product WHERE Product_ID=?'
        cur.execute(delete, (product_id.value,))
       
        # Load the data into a DataFrame
        #results.to_sql("Product", con, if_exists="replace", index=False, index_label='Product_ID')
        full_table = pd.read_sql_query("SELECT * from product", con)
        table.value = full_table
    #delete_query.value = f'SELECT {product_id.value} from Product'
    
    elif table_options.value == 'Customer':
        delete = {'Customer_ID': product_id.value}
        full_table = pd.read_sql_query("SELECT * from Customer", con)
    
        df = pd.DataFrame.from_dict([delete])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Customer_ID'], keep=False).reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Customer", con, if_exists="replace", index=False, index_label='Customer_ID')
        table.value = results
    
    elif table_options.value == 'Employees':
        delete = {'Employee_ID': product_id.value}
        full_table = pd.read_sql_query("SELECT * from Employees", con)
    
        df = pd.DataFrame.from_dict([delete])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Employee_ID'], keep=False).reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Employees", con, if_exists="replace", index=False, index_label='Employee_ID')
        table.value = results
        
    elif table_options.value == 'Orders':
        delete = {'Order_ID': product_id.value}
        full_table = pd.read_sql_query("SELECT * from Orders", con)
    
        df = pd.DataFrame.from_dict([delete])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Order_ID'], keep=False).reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Orders", con, if_exists="replace", index=False, index_label='Order_ID')
        table.value = results
delete_button.on_click(delete_event)


Watcher(inst=Button(button_type='danger', name='Delete entry'), cls=<class 'panel.widgets.button.Button'>, fn=<function delete_event at 0x7fabd3ed9040>, mode='args', onlychanged=False, parameter_names=('clicks',), what='value', queued=False, precedence=0)

In [ ]:
def update_event(event):
    con = sqlite3.connect("DBMS.db")
    if table_options.value == 'Product':
    
        update = {'Product_ID': product_id.value, 'Product_Name': product_name.value, 'Vendor_Cost': vendor_cost.value, 'Customer_Cost': (vendor_cost.value + (vendor_cost.value * 5 / 100))}
        full_table = pd.read_sql_query("SELECT * from product", con)
    
        df = pd.DataFrame.from_dict([update])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Product_ID'], keep='last').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Product", con, if_exists="replace", index=False, index_label='Product_ID')
        table.value = results
        
    elif table_options.value == 'Customer':
        update = {'Customer_ID': customer_id.value, 'First_Name': first_name.value, 'Last_Name': last_name.value, 'Phone': Phone_C.value,
                'City': city.value, 'Zipcode': zip_code.value}
        full_table = pd.read_sql_query("SELECT * from Customer", con)
    
        df = pd.DataFrame.from_dict([update])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Customer_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Customer", con, if_exists="replace", index=False, index_label='Customer_ID')
        table.value = results
        full_table = pd.read_sql_query("SELECT * from Customer", con)
        
    elif table_options.value == 'Employees':
        update = {'Employee_ID': employee_id.value, 'First_Name': FirstName.value, 'Last_Name': LastName.value, 'Phone': Phone_E.value,
                'Job_title': Job_title.value}
        full_table = pd.read_sql_query("SELECT * from Employees", con)
    
        df = pd.DataFrame.from_dict([update])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Employee_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Employees", con, if_exists="replace", index=False, index_label='Employee_ID')
        table.value = results
        full_table
    
    elif table_options.value == 'Orders':
        update = {'Order_ID': order_id.value, 'Product_ID': product_id.value, 'Customer_ID': customer_id.value, 'Employee_ID': employee_id.value,
                'Order_date': order_date.value, 'Quantity': quantity.value}
        full_table = pd.read_sql_query("SELECT * from Orders", con)
    
        df = pd.DataFrame.from_dict([update])
    
        results = pd.concat([full_table, df]).drop_duplicates(subset=['Order_ID'], keep='first').reset_index(drop=True)

        # Load the data into a DataFrame
        results.to_sql("Orders", con, if_exists="replace", index=False, index_label='Order_ID')
        table.value = results
        full_table = pd.read_sql_query("SELECT * from Orders", con)
    #update_query.value = f'clicked {update_button.clicks} times'
    
update_button.on_click(update_event)

In [12]:
title = pn.Row(Title_button)
inputs = pn.Row(product, customer,employees, orders)
queries = pn.Column(#predefined_query,
                    pn.Row(table_options, 
                    pn.Column(insert_button),
                    pn.Column(delete_button), 
                    pn.Column(update_button)))
ui = pn.Column(title,inputs, queries, table)
ui

Column
    [0] Row
        [0] Button(button_type='primary', name='Jewelry Business')
    [1] Row
        [0] Column
            [0] IntInput(name='Product ID')
            [1] TextInput(name='Product Name')
            [2] FloatInput(name='Vendor Cost')
            [3] FloatInput(name='Customer Cost')
        [1] Column
            [0] IntInput(name='Customer ID')
            [1] TextInput(name='First Name')
            [2] TextInput(name='Last Name')
            [3] TextInput(name='Phone Number')
            [4] TextInput(name='City')
            [5] IntInput(name='Zip Code')
        [2] Column
            [0] IntInput(name='Employee ID')
            [1] TextInput(name='First Name')
            [2] TextInput(name='Last Name')
            [3] TextInput(name='Phone Number')
            [4] TextInput(name='Job Title')
        [3] Column
            [0] IntInput(name='Order ID')
            [1] IntInput(name='Product ID')
            [2] IntInput(name='Customer ID')
            [3] IntInput(name='Employee ID')
            [4] DatePicker(name='Order Date')
            [5] IntInput(name='Quantity')
    [2] Column
        [0] Row
            [0] RadioBoxGroup(options=['Product', 'Customer', ...], value='Product')
            [1] Column
                [0] Button(button_type='success', name='Insert entry')
            [2] Column
                [0] Button(button_type='danger', name='Delete entry')
            [3] Column
                [0] Button(button_type='warning', name='Update entry')
    [3] DataFrame(name='Table', value=   Product_ID P...)

In [13]:
#ui.servable().show()